In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from skl2onnx.helpers.onnx_helper import select_model_inputs_outputs
from skl2onnx.helpers.onnx_helper import save_onnx_model
from skl2onnx.helpers.onnx_helper import enumerate_model_node_outputs
from skl2onnx.helpers.onnx_helper import load_onnx_model

from stitchnet.stitchonnx.utils import change_input_dim, get_fragments, Net

import numpy as np
from tqdm import tqdm
from glob import glob
from stitchnet.finetuning.finetune import finetune
import os
import torch

num_classes = 2
num_epochs = 1
feature_extract = True
batch_size = 256

modelnames = sorted(glob('_models/*.onnx'))

os.makedirs('_results/finetune', exist_ok=True)
for i,modelname in tqdm(enumerate(modelnames[3:])):
    model_onnx1 = load_onnx_model(modelname)
    model_ft, hist = finetune(model_onnx1, num_classes, num_epochs, batch_size, feature_extract)
    name = os.path.basename(modelname)
    namewithoutext = os.path.splitext(name)[0]
    model_ft = model_ft.cpu()
    torch.onnx.export(model_ft, torch.ones(1,3,224,224), f'_results/finetune/{namewithoutext}_ft.onnx')
    with open(f'_results/finetune/{namewithoutext}.txt', 'w') as f:
        for line in hist:
            f.write(f'{",".join([f"{x:.4}" for x in line])}\n')   
    # break

/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch Version:  1.12.1
Torchvision Version:  0.13.1+cu102
device cuda:0


0it [00:00, ?it/s]

Params to learn:
	 Gemm_0.weight
	 Gemm_0.bias
Epoch 0/0
----------



100%|██████████| 78/78 [00:24<00:00,  3.24it/s]


...


100%|██████████| 78/78 [00:33<00:00,  2.33it/s]


train Loss: 0.1006 Acc: 0.9622



100%|██████████| 20/20 [00:10<00:00,  1.93it/s]


val Loss: 0.0466 Acc: 0.9829

Training complete in 0m 44s
Best val Acc: 0.982908


2it [01:28, 44.01s/it]
